In [31]:
import pandas as pd
import numpy as np
import plotly.express as px
from sklearn.preprocessing import LabelBinarizer

In [166]:
import statsmodels.formula.api as smf

In [194]:
ORIGINAL = pd.read_csv('data/survey_results_public_mega_inc.csv')

/home/jvanderwolf/projects/school/codam/events/no-bias-no-cry/.venv/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning:

Columns (11,12,13,25,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61) have mixed types.Specify dtype option on import or set low_memory=False.



In [224]:
df = ORIGINAL.copy()

In [225]:
VARS_REQUIRED = [
    'ConvertedSalary',
    'JobSatisfaction',
    'CareerSatisfaction',
    'SexualOrientation',
    'DevType',
    'EducationTypes',
    'YearsCoding',
    'YearsCodingProf',
    'Gender'
]

In [226]:
df_filter_country = df[(df['Country'] == 'Netherlands') | (df['Country'] == 'Germany')]
df_filter_employ = df_filter_country[(df_filter_country['Employment'] == 'Employed full-time')]
df_filter_salary = df_filter_employ[VARS_REQUIRED]

In [227]:
len(df_filter_salary)

5561

In [228]:
# TEMPORARY
df_filter_salary = df_filter_salary[(df_filter_salary['Gender'].notnull()) & (df_filter_salary['SexualOrientation'].notnull())]

In [229]:
# clean up np.NaN values
df_filter_salary['GenderBinary'] = np.where((df_filter_salary['Gender'] == 'Male') | (df_filter_salary['Gender'] == 'Female') | (df_filter_salary['Gender'] == 'Female;Male'), 'GenderConforming', 'GenderNonconforming')
df_filter_salary['Male'] = np.where((df_filter_salary['Gender'] == 'Male'), 1, 0)
df_filter_salary['OrientationBinary'] = np.where((df_filter_salary['SexualOrientation'] == 'Straight or heterosexual'), 'OrientConforming', 'OrientNonConforming')

In [230]:
filtered = df_filter_salary.copy()

In [231]:
filtered = pd.concat([filtered, pd.get_dummies(filtered['GenderBinary'])], axis=1)
filtered = pd.concat([filtered, pd.get_dummies(filtered['OrientationBinary'])], axis=1)
filtered = pd.concat([filtered, pd.get_dummies(filtered['Gender'])], axis=1)
filtered = pd.concat([filtered, pd.get_dummies(filtered['SexualOrientation'])], axis=1)

In [234]:
final_df = filtered[['ConvertedSalary', 'OrientationBinary', 'Male', 'YearsCodingProf', 'GenderBinary']]
results = smf.ols('ConvertedSalary ~ OrientationBinary + Male + YearsCodingProf + GenderBinary', data=final_df).fit()